In [1]:
import BiLSTM
import numpy as np
import random
import spacy
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchtext import data
from torchtext import datasets
from torchtext.data.utils import get_tokenizer

In [2]:
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
# Ensures that the same sequence of random numbers will be generated every time.
# Sets the seed for generating random numbers in a torch.Generator object.

In [3]:
tokenizer = get_tokenizer('spacy', language='en_core_web_sm')

In [4]:

TEXT = data.Field(tokenize=tokenizer, lower = True)
LABEL = data.LabelField()
print(TEXT.dtype)
print(LABEL)

torch.int64


In [5]:
train_data, valid_data, test_data = datasets.SNLI.splits(TEXT, LABEL)
#loading data from the SNLI dataset

downloading snli_1.0.zip


snli_1.0.zip: 100%|██████████| 94.6M/94.6M [02:00<00:00, 784kB/s] 


extracting


In [6]:
MIN_FREQ = 2

TEXT.build_vocab(train_data, 
                 min_freq = MIN_FREQ,
                 vectors = "glove.6B.300d",
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)
#building the vocabulary for the text and label fields

.vector_cache/glove.6B.zip: 862MB [09:22, 1.53MB/s]                               
100%|█████████▉| 399999/400000 [00:22<00:00, 17631.70it/s]


In [ ]:
BATCH_SIZE = 512
CUDA_LAUNCH_BLOCKING=1
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 300
N_LSTM_LAYERS = 2
N_FC_LAYERS = 3
OUTPUT_DIM = len(LABEL.vocab)
DROPOUT = 0.25
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
N_EPOCHS = 10


In [7]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}"
        f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}"
        f"Debug: These two lines should show be the same words (for TEXT) \n{TEXT.vocab.freqs.most_common(20)}"
        f"{TEXT.vocab.itos[:20]}" #showing the most common words in the vocabulary good if they are the same as the above frequency list

        f"Debug: These two lines should show be the same words (for LABEL){LABEL.vocab.freqs.most_common()}" #showing the frequency of the labels in the vocabulary
        f"{LABEL.vocab.itos}"
        )


Unique tokens in TEXT vocabulary: 23551Unique tokens in LABEL vocabulary: 3Debug: These two lines should show be the same words (for TEXT) 
[('a', 1438991), ('.', 962527), ('the', 534692), ('in', 407296), ('is', 373543), ('man', 266236), ('on', 235904), ('and', 206363), ('are', 199114), ('of', 192428), ('with', 169236), ('woman', 137630), ('two', 122259), ('people', 121154), (',', 114331), ('to', 113972), ('at', 98656), ('wearing', 81024), ('an', 80212), ('his', 72467)]['<unk>', '<pad>', 'a', '.', 'the', 'in', 'is', 'man', 'on', 'and', 'are', 'of', 'with', 'woman', 'two', 'people', ',', 'to', 'at', 'wearing']Debug: These two lines should show be the same words (for LABEL)[('entailment', 183416), ('contradiction', 183187), ('neutral', 182764)]['entailment', 'contradiction', 'neutral']


In [10]:


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)
# creating iterators for the data 
print(f"{device}, {'gpu' if device == torch.device('cuda') else 'slow training on cpu'}")


cuda, gpu


In [11]:



model = BiLSTM.NLIBiLSTM(INPUT_DIM,
                  EMBEDDING_DIM,
                  HIDDEN_DIM,
                  N_LSTM_LAYERS,
                  N_FC_LAYERS,
                  OUTPUT_DIM,
                  DROPOUT,
                  PAD_IDX).to(device)
# creating the model
#print(model.sumary())

In [12]:
pretrained_embeddings = TEXT.vocab.vectors
# getting the pretrained embeddings
print(pretrained_embeddings.shape)

torch.Size([23551, 300])


In [13]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ..., -1.4447,  0.8402, -0.8668],
        [ 0.1032, -1.6268,  0.5729,  ...,  0.3180, -0.1626, -0.0417],
        [-0.2971,  0.0940, -0.0967,  ...,  0.0597, -0.2285,  0.2960],
        ...,
        [-0.2149,  0.0846, -0.2949,  ...,  0.2379,  0.4804, -0.3348],
        [-1.0809, -1.1397, -0.9308,  ..., -1.6457,  0.5161, -1.6925],
        [ 0.3580, -0.0304,  0.3355,  ...,  0.0703, -0.5158,  0.1819]],
       device='cuda:0')

In [14]:
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)
# setting the padding index to zero
print(model.embedding.weight.data)

tensor([[-0.1117, -0.4966,  0.1631,  ..., -1.4447,  0.8402, -0.8668],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.2971,  0.0940, -0.0967,  ...,  0.0597, -0.2285,  0.2960],
        ...,
        [-0.2149,  0.0846, -0.2949,  ...,  0.2379,  0.4804, -0.3348],
        [-1.0809, -1.1397, -0.9308,  ..., -1.6457,  0.5161, -1.6925],
        [ 0.3580, -0.0304,  0.3355,  ...,  0.0703, -0.5158,  0.1819]],
       device='cuda:0')


In [15]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {count_parameters(model): } trainable parameters')
tempcount_parameters=count_parameters(model)

The model has  15092403 trainable parameters


In [16]:
model.embedding.weight.requires_grad = False
# setting the embedding layer to not be trainable

In [23]:
print(f'The model has {count_parameters(model):} unfrozen trainable parameters and we froze {(tempcount_parameters-count_parameters(model)):,}' )


The model has  8027103 unfrozen trainable parameters and we froze 7,065,300


In [24]:
optimizer = optim.AdamW(model.parameters(),)

In [25]:
criterion = nn.CrossEntropyLoss().to(device)

In [26]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.cuda.FloatTensor([y.shape[0]])

In [27]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        prem = batch.premise
        hypo = batch.hypothesis
        labels = batch.label
        
        optimizer.zero_grad()
        
        #prem = [prem sent len, batch size]
        #hypo = [hypo sent len, batch size]
        
        predictions = model(prem, hypo)
        
        #predictions = [batch size, output dim]
        #labels = [batch size]
        
        loss = criterion(predictions, labels)
                
        acc = categorical_accuracy(predictions, labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [28]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            prem = batch.premise
            hypo = batch.hypothesis
            labels = batch.label
                        
            predictions = model(prem, hypo)
            
            loss = criterion(predictions, labels)
                
            acc = categorical_accuracy(predictions, labels)
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [29]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [30]:
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 4m 58s
	Train Loss: 0.770 | Train Acc: 65.80%
	 Val. Loss: 0.668 |  Val. Acc: 72.12%
Epoch: 02 | Epoch Time: 4m 59s
	Train Loss: 0.630 | Train Acc: 73.90%
	 Val. Loss: 0.606 |  Val. Acc: 75.03%
Epoch: 03 | Epoch Time: 5m 0s
	Train Loss: 0.573 | Train Acc: 76.78%
	 Val. Loss: 0.558 |  Val. Acc: 77.30%
Epoch: 04 | Epoch Time: 5m 1s
	Train Loss: 0.533 | Train Acc: 78.68%
	 Val. Loss: 0.539 |  Val. Acc: 78.19%
Epoch: 05 | Epoch Time: 5m 7s
	Train Loss: 0.502 | Train Acc: 80.14%
	 Val. Loss: 0.523 |  Val. Acc: 78.89%
Epoch: 06 | Epoch Time: 5m 7s
	Train Loss: 0.474 | Train Acc: 81.40%
	 Val. Loss: 0.504 |  Val. Acc: 79.80%
Epoch: 07 | Epoch Time: 5m 9s
	Train Loss: 0.448 | Train Acc: 82.54%
	 Val. Loss: 0.501 |  Val. Acc: 80.00%
Epoch: 08 | Epoch Time: 5m 7s
	Train Loss: 0.423 | Train Acc: 83.62%
	 Val. Loss: 0.495 |  Val. Acc: 80.84%
Epoch: 09 | Epoch Time: 5m 7s
	Train Loss: 0.397 | Train Acc: 84.74%
	 Val. Loss: 0.504 |  Val. Acc: 80.76%
Epoch: 10 | Epoch Time: 5m

In [32]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.513 |  Test Acc: 80.17%


In [33]:
def predict_inference(premise, hypothesis, text_field, label_field, model, device):
    
    model.eval()
    
    if isinstance(premise, str):
        premise = text_field.tokenize(premise)
    
    if isinstance(hypothesis, str):
        hypothesis = text_field.tokenize(hypothesis)
    
    if text_field.lower:
        premise = [t.lower() for t in premise]
        hypothesis = [t.lower() for t in hypothesis]
        
    premise = [text_field.vocab.stoi[t] for t in premise]
    hypothesis = [text_field.vocab.stoi[t] for t in hypothesis]
    
    premise = torch.LongTensor(premise).unsqueeze(1).to(device)
    hypothesis = torch.LongTensor(hypothesis).unsqueeze(1).to(device)
    
    prediction = model(premise, hypothesis)
    
    prediction = prediction.argmax(dim=-1).item()
    
    return label_field.vocab.itos[prediction]

In [34]:
premise = 'a man sitting on a green bench.'
hypothesis = 'a woman sitting on a green bench.'
for i in range(750):
    predict_inference(premise, hypothesis, TEXT, LABEL, model, device)

print("done")

done


In [35]:
premise = 'Move the sun to the right and above.'
hypothesis = 'A user wants to increase the sound.'

predict_inference(premise, hypothesis, TEXT, LABEL, model, device)

'neutral'

In [38]:
premise = 'Move the sun to the right'
hypothesis = 'A user wants the sun to move to the right.'

predict_inference(premise, hypothesis, TEXT, LABEL, model, device)

'neutral'

In [37]:
premise = 'a man sitting on a green bench.'
hypothesis = 'a person on a park bench'

predict_inference(premise, hypothesis, TEXT, LABEL, model, device)

'neutral'

In [41]:
#fitetuningpretrained pytorch from scratch
#del model
del criterion
#del batch
torch.cuda.empty_cache()